<a href="https://colab.research.google.com/github/nhareesha/MLAI/blob/main/LLM/finetuned/DataSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
import re  # Import the `re` module

In [ ]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Define the DDLs
ddl = """
CREATE TABLE customers (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  email VARCHAR(255)
);

CREATE TABLE orders (
  id INT PRIMARY KEY,
  customer_id INT,
  order_date DATE,
  FOREIGN KEY (customer_id) REFERENCES customers(id)
);
"""

# Define the sample data
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 1, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2, 'order_date': '2022-01-15'}
    ]
}

In [ ]:
# Create a DataFrame for the sample data
df = pd.DataFrame(sample_data['customers'] + sample_data['orders'])
print(df)


   id      name                 email  customer_id  order_date
0   1  John Doe  john.doe@example.com          NaN         NaN
1   2  Jane Doe  jane.doe@example.com          NaN         NaN
2   1       NaN                   NaN          1.0  2022-01-01
3   2       NaN                   NaN          2.0  2022-01-15


In [ ]:
# Define the sequence-to-sequence model
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
# Define the custom dataset class
class DDLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def something(self, idx):
        row = self.df.iloc[idx]
        print("Printing ROW")
        print(row)
        print(row['table_name'])
        input_text = f"CREATE TABLE {row['table_name']} ({row['column_name']} {row['data_type']})"
        output_text = row['sample_data']
        inputs = self.tokenizer(input_text, return_tensors='pt')
        labels = self.tokenizer(output_text, return_tensors='pt')
        return {'input_ids': inputs['input_ids'].flatten(), 'labels': labels['input_ids'].flatten()}

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # print("Printing ROW")
        # print(row)
        # Determine the table name based on the index
        if idx < len(sample_data['customers']):
            table_name = 'customers'
            column_name = 'id'
            data_type = 'INT'
            input_text = f"CREATE TABLE {table_name} ({column_name} {data_type})"
            output_text = f"INSERT INTO {table_name} ({column_name}) VALUES ({row['id']})"
        else:
            table_name = 'orders'
            column_name = 'id'
            data_type = 'INT'
            input_text = f"CREATE TABLE {table_name} ({column_name} {data_type})"
            output_text = f"INSERT INTO {table_name} ({column_name}) VALUES ({row['id']})"

        inputs = self.tokenizer(input_text, return_tensors='pt')
        labels = self.tokenizer(output_text, return_tensors='pt')
        return {'input_ids': inputs['input_ids'].flatten(), 'labels': labels['input_ids'].flatten()}


In [ ]:
# Create the dataset and data loader
dataset = DDLDataset(df, tokenizer)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss() # Create an instance of LossFunction
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(100):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels)
        # print(type(outputs))
        # loss = criterion(outputs, labels) # Needs to be a Tensor.
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')
    model.eval()


Epoch 1, Loss: 4.685617446899414
Epoch 2, Loss: 4.09550666809082
Epoch 3, Loss: 3.8358912467956543
Epoch 4, Loss: 3.6946582794189453
Epoch 5, Loss: 3.5764377117156982
Epoch 6, Loss: 3.4286916255950928
Epoch 7, Loss: 3.4149281978607178
Epoch 8, Loss: 3.5131187438964844
Epoch 9, Loss: 3.45695424079895
Epoch 10, Loss: 3.0852248668670654
Epoch 11, Loss: 3.076669454574585
Epoch 12, Loss: 2.892866849899292
Epoch 13, Loss: 3.081608295440674
Epoch 14, Loss: 2.932990074157715
Epoch 15, Loss: 2.8433403968811035
Epoch 16, Loss: 2.6560235023498535
Epoch 17, Loss: 2.620180606842041
Epoch 18, Loss: 2.47871732711792
Epoch 19, Loss: 2.657715320587158
Epoch 20, Loss: 2.4728803634643555
Epoch 21, Loss: 2.499300003051758
Epoch 22, Loss: 2.3772404193878174
Epoch 23, Loss: 2.2535347938537598
Epoch 24, Loss: 2.220264434814453
Epoch 25, Loss: 2.2212698459625244
Epoch 26, Loss: 2.204831123352051
Epoch 27, Loss: 2.1427695751190186
Epoch 28, Loss: 2.1274654865264893
Epoch 29, Loss: 2.4987714290618896
Epoch 30, 

In [ ]:
# Use the trained model to generate sample data
def generate_sample_data_1(input_text):
    inputs = tokenizer(input_text, return_tensors='pt')
    print(inputs)
    outputs = model.generate(inputs['input_ids'], max_length=50)
    print(outputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import faker

def generate_sample_data(input_text):
    fake = faker.Faker()
    inputs = tokenizer(input_text, return_tensors='pt')
    outputs = model.generate(inputs['input_ids'], max_length=50)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the table name and column names from the input text
    table_name = input_text.split('(')[0].split()[-1]
    column_names = input_text.split('(')[1].split(')')[0].split(',')

    # Generate sample data using faker
    sample_data = []
    for _ in range(10):  # Generate 10 rows of sample data
        row = {}
        for column_name in column_names:
            column_name = column_name.strip()
            if 'id' in column_name:
                row[column_name] = fake.random_int()
            elif 'name' in column_name:
                row[column_name] = fake.name()
            elif 'DATE' in column_name:
                row[column_name] = fake.date()
            elif 'email' in column_name:
                row[column_name] = fake.email()
        sample_data.append(row)

    return output_text, sample_data

In [ ]:
input_text = "CREATE TABLE customers (id INT PRIMARY KEY, name VARCHAR(255), email VARCHAR(255))"
sample_data = generate_sample_data(input_text)
print(sample_data)

('INSERT INTO customers (id, name VARCHAR(255)), email VARCHAR(255))', [{'id INT PRIMARY KEY': 2807, 'name VARCHAR': 'Julie Rodgers'}, {'id INT PRIMARY KEY': 3289, 'name VARCHAR': 'Daniel Johnson'}, {'id INT PRIMARY KEY': 3964, 'name VARCHAR': 'Tiffany Jennings'}, {'id INT PRIMARY KEY': 8647, 'name VARCHAR': 'Lauren Ward DDS'}, {'id INT PRIMARY KEY': 8338, 'name VARCHAR': 'Timothy Patterson'}, {'id INT PRIMARY KEY': 5167, 'name VARCHAR': 'Adam Young'}, {'id INT PRIMARY KEY': 3197, 'name VARCHAR': 'Autumn Underwood'}, {'id INT PRIMARY KEY': 2918, 'name VARCHAR': 'David Johnson'}, {'id INT PRIMARY KEY': 9459, 'name VARCHAR': 'Diane Stewart'}, {'id INT PRIMARY KEY': 123, 'name VARCHAR': 'Diane Reynolds'}])


```
# This is formatted as code

import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Define the DDLs
ddl = """
CREATE TABLE customers (
  id INT PRIMARY KEY,
  name VARCHAR(255),
  email VARCHAR(255)
);

CREATE TABLE orders (
  id INT PRIMARY KEY,
  customer_id INT,
  order_date DATE,
  FOREIGN KEY (customer_id) REFERENCES customers(id)
);
"""

# Define the sample data
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 1, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2, 'order_date': '2022-01-15'}
    ]
}

# Create a DataFrame for the sample data
df = pd.DataFrame(sample_data['customers'] + sample_data['orders'])
print(df)
# Define the sequence-to-sequence model
model_name = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the custom dataset class
class DDLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def something(self, idx):
        row = self.df.iloc[idx]
        print("Printing ROW")
        print(row)
        print(row['table_name'])
        input_text = f"CREATE TABLE {row['table_name']} ({row['column_name']} {row['data_type']})"
        output_text = row['sample_data']
        inputs = self.tokenizer(input_text, return_tensors='pt')
        labels = self.tokenizer(output_text, return_tensors='pt')
        return {'input_ids': inputs['input_ids'].flatten(), 'labels': labels['input_ids'].flatten()}
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        print("Printing ROW")
        print(row)        
        # Determine the table name based on the index
        if idx < len(sample_data['customers']):
            table_name = 'customers'
            column_name = 'id'
            data_type = 'INT'
            input_text = f"CREATE TABLE {table_name} ({column_name} {data_type})"
            output_text = f"INSERT INTO {table_name} ({column_name}) VALUES ({row['id']})"
        else:
            table_name = 'orders'
            column_name = 'id'
            data_type = 'INT'
            input_text = f"CREATE TABLE {table_name} ({column_name} {data_type})"
            output_text = f"INSERT INTO {table_name} ({column_name}) VALUES ({row['id']})"
    
        inputs = self.tokenizer(input_text, return_tensors='pt')
        labels = self.tokenizer(output_text, return_tensors='pt')
        return {'input_ids': inputs['input_ids'].flatten(), 'labels': labels['input_ids'].flatten()}

# Create the dataset and data loader
dataset = DDLDataset(df, tokenizer)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss() # Create an instance of LossFunction
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels)
        print(type(outputs))
        # loss = criterion(outputs, labels) # Needs to be a Tensor.
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')
    model.eval()

# Use the trained model to generate sample data
def generate_sample_data(input_text):
    inputs = tokenizer(input_text, return_tensors='pt')
    outputs = model.generate(inputs['input_ids'], max_length=2000)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

input_text = "CREATE TABLE customers (id INT PRIMARY KEY, name VARCHAR(255), email VARCHAR(255))"
sample_data = generate_sample_data(input_text)
print(sample_data)


```



In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader

# Define the sample data
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 1, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2, 'order_date': '2022-01-15'}
    ]
}

# Create DataFrames for the sample data
customers_df = pd.DataFrame(sample_data['customers'])
orders_df = pd.DataFrame(sample_data['orders'])

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2-medium'  # You can use 'gpt2', 'gpt2-medium', 'gpt2-large', etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Fine-tune the tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token for GPT-2

# Custom dataset class
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Generate input text based on the table data
        if 'name' in row and 'email' in row:
            input_text = f"INSERT INTO customers (id, name, email) VALUES ({row['id']}, '{row['name']}', '{row['email']}');"
        else:
            input_text = f"INSERT INTO orders (id, customer_id, order_date) VALUES ({row['id']}, {row['customer_id']}, '{row['order_date']}');"

        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
        return {'input_ids': inputs['input_ids'].squeeze(), 'attention_mask': inputs['attention_mask'].squeeze()}

# Create the dataset and data loader
combined_df = pd.concat([customers_df, orders_df], ignore_index=True)
dataset = SQLDataset(combined_df, tokenizer)

# Create a collate function to handle padding dynamically
def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]

    # Pad sequences dynamically
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_masks}

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(60):  # You can increase the number of epochs
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

# Use the trained model to generate sample data
def generate_sql_data(prompt):
    inputs = tokenizer(prompt, return_tensors='pt')
    inputs = {key: val.to(device) for key, val in inputs.items()}
    outputs = model.generate(**inputs, max_length=100, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example SQL generation
input_prompt = "INSERT INTO customers (id, name, email) VALUES"
sample_sql = generate_sql_data(input_prompt)
print(sample_sql)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1, Loss: 2.2246330976486206
Epoch 2, Loss: 1.4954220652580261
Epoch 3, Loss: 2.4478172063827515
Epoch 4, Loss: 2.1031181812286377
Epoch 5, Loss: 0.7138573229312897
Epoch 6, Loss: 0.6366875469684601
Epoch 7, Loss: 1.3573296070098877
Epoch 8, Loss: 1.1255963444709778
Epoch 9, Loss: 0.4683581292629242
Epoch 10, Loss: 0.46456393599510193
Epoch 11, Loss: 0.9048112630844116
Epoch 12, Loss: 0.7713034152984619
Epoch 13, Loss: 0.5606061071157455
Epoch 14, Loss: 0.5143173635005951
Epoch 15, Loss: 0.39989039301872253
Epoch 16, Loss: 0.19277561455965042
Epoch 17, Loss: 0.23095811158418655
Epoch 18, Loss: 0.10581569001078606
Epoch 19, Loss: 0.12392636016011238
Epoch 20, Loss: 0.0988464504480362
Epoch 21, Loss: 0.112318255007267
Epoch 22, Loss: 0.11788876354694366
Epoch 23, Loss: 0.11346694827079773
Epoch 24, Loss: 0.23099640756845474
Epoch 25, Loss: 0.16534792259335518
Epoch 26, Loss: 0.09857083112001419
Epoch 27, Loss: 0.07259278744459152
Epoch 28, Loss: 0.0720802191644907
Epoch 29, Loss: 0.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 60, Loss: 0.1109996810555458
INSERT INTO customers (id, name, email) VALUES (1, 'nan', 'nan');


Modified - Using GPT model

In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from faker import Faker

# Initialize Faker
fake = Faker()

# Sample data with placeholders
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 1, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2, 'order_date': '2022-01-15'}
    ]
}

# Create DataFrames for the sample data
customers_df = pd.DataFrame(sample_data['customers'])
orders_df = pd.DataFrame(sample_data['orders'])

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token for GPT-2

class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        if 'name' in row and 'email' in row:
            input_text = f"INSERT INTO customers (id, name, email) VALUES ({row['id']}, '{fake.name()}', '{fake.email()}');"
        else:
            input_text = f"INSERT INTO orders (id, customer_id, order_date) VALUES ({row['id']}, {row['customer_id']}, '{row['order_date']}');"

        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
        return {'input_ids': inputs['input_ids'].squeeze(), 'attention_mask': inputs['attention_mask'].squeeze()}

# Combine datasets and create DataLoader with padding
combined_df = pd.concat([customers_df, orders_df], ignore_index=True)
dataset = SQLDataset(combined_df, tokenizer)

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_masks}

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 60
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.clone()  # GPT-2 generates the next token

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader)}')

model.eval()

# Generate SQL statements
def generate_sql_data_from_trained_model(prompt, num_statements=20):
    generated_statements = []
    for _ in range(num_statements):
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], max_length=100, num_return_sequences=1)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Post-process to replace 'nan' or invalid data with realistic fake data
        generated_sql = generated_sql.replace("'nan'", f"'{fake.name()}'")
        generated_sql = generated_sql.replace("'nan'", f"'{fake.email()}'")
        generated_statements.append(generated_sql)

    return generated_statements

# Generate unique SQL statements
def generate_sql_data_not_generalized(num_statements=20):
    generated_statements = []
    for _ in range(num_statements):
        # Generate unique fake data for each statement
        id_value = fake.random_int(min=1, max=1000)
        name_value = fake.name()
        email_value = fake.email()

        generated_sql = f"INSERT INTO customers (id, name, email) VALUES ({id_value}, '{name_value}', '{email_value}');"
        generated_statements.append(generated_sql)

    return generated_statements

import re

def generate_sql_data(prompt, num_statements=20):
    generated_statements = []
    for _ in range(num_statements):
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        # Create attention mask
        inputs['attention_mask'] = (inputs['input_ids'] != tokenizer.pad_token_id).long()

        outputs = model.generate(inputs['input_ids'], max_length=100, num_return_sequences=1)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Define patterns to identify column types
        name_pattern = re.compile(r"\bname\b", re.IGNORECASE)
        email_pattern = re.compile(r"\bemail\b", re.IGNORECASE)
        date_pattern = re.compile(r"\bdate\b", re.IGNORECASE)
        id_pattern = re.compile(r"\bid\b", re.IGNORECASE)

        # Post-process to replace 'nan' or invalid data with realistic fake data
        def replace_with_fake_data(match):
            column_name = match.group(0).lower()
            if name_pattern.search(column_name):
                return f"'{fake.name()}'"
            elif email_pattern.search(column_name):
                return f"'{fake.email()}'"
            elif date_pattern.search(column_name):
                return f"'{fake.date()}'"
            elif id_pattern.search(column_name):
                return str(fake.random_int(min=1, max=1000))
            else:
                return f"'{fake.word()}'"  # Generic replacement for other columns

        # Replace any 'nan' or invalid data by matching with column names
        generated_sql = re.sub(r"'nan'", replace_with_fake_data, generated_sql)
        generated_sql = re.sub(r"'\d+'", replace_with_fake_data, generated_sql)

        generated_statements.append(generated_sql)

    return generated_statements




Epoch 1/60, Loss: 3.001787543296814
Epoch 2/60, Loss: 2.599470615386963
Epoch 3/60, Loss: 2.19442880153656
Epoch 4/60, Loss: 1.9167596101760864
Epoch 5/60, Loss: 1.8824473023414612
Epoch 6/60, Loss: 2.0172500014305115
Epoch 7/60, Loss: 1.7893186211585999
Epoch 8/60, Loss: 1.5327982902526855
Epoch 9/60, Loss: 1.4570062756538391
Epoch 10/60, Loss: 1.315062701702118
Epoch 11/60, Loss: 1.4665381908416748
Epoch 12/60, Loss: 1.3962971568107605
Epoch 13/60, Loss: 1.6835548877716064
Epoch 14/60, Loss: 1.307212233543396
Epoch 15/60, Loss: 1.3668845891952515
Epoch 16/60, Loss: 1.3635870814323425
Epoch 17/60, Loss: 1.431712031364441
Epoch 18/60, Loss: 1.2830909490585327
Epoch 19/60, Loss: 1.2621662616729736
Epoch 20/60, Loss: 1.2991197109222412
Epoch 21/60, Loss: 1.165330946445465
Epoch 22/60, Loss: 1.2258515357971191
Epoch 23/60, Loss: 1.240250825881958
Epoch 24/60, Loss: 1.5886485576629639
Epoch 25/60, Loss: 1.1873716115951538
Epoch 26/60, Loss: 1.2995360493659973
Epoch 27/60, Loss: 1.242852032

In [ ]:
# Example usage:
prompt = "INSERT INTO customers (id, name, email) VALUES (1, 'nan', 'nan');"
sample_sql_statements = generate_sql_data(prompt, num_statements=20)

# Print the generated SQL statements
for statement in sample_sql_statements:
    print(statement)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

INSERT INTO customers (id, name, email) VALUES (1, 'such', 'be');
INSERT INTO customers (id, name, email) VALUES (1, 'another', 'or');
INSERT INTO customers (id, name, email) VALUES (1, 'participant', 'approach');
INSERT INTO customers (id, name, email) VALUES (1, 'magazine', 'number');
INSERT INTO customers (id, name, email) VALUES (1, 'important', 'receive');
INSERT INTO customers (id, name, email) VALUES (1, 'right', 'attention');
INSERT INTO customers (id, name, email) VALUES (1, 'specific', 'nature');
INSERT INTO customers (id, name, email) VALUES (1, 'with', 'start');
INSERT INTO customers (id, name, email) VALUES (1, 'dog', 'report');
INSERT INTO customers (id, name, email) VALUES (1, 'dark', 'six');
INSERT INTO customers (id, name, email) VALUES (1, 'last', 'somebody');
INSERT INTO customers (id, name, email) VALUES (1, 'while', 'can');
INSERT INTO customers (id, name, email) VALUES (1, 'morning', 'store');
INSERT INTO customers (id, name, email) VALUES (1, 'quickly', 'speak');

In [ ]:
# Example usage:
prompt = "INSERT INTO CUSTOMERS (id, fname,lname, email) Values"
sample_sql_statements = generate_sql_data(prompt, num_statements=20)

# Print the generated SQL statements
for statement in sample_sql_statements:
    print(statement)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 'joseph@example.org');
INSERT INTO person (id, fname,lname, email) Values (1, 'David Smith', 

In [ ]:
### Another type of prompt
input_prompt = "INSERT INTO customers (id, name, email) VALUES"
sample_sql_statements = generate_sql_data(input_prompt, num_statements=20)

# Print generated SQL statements
for statement in sample_sql_statements:
    print(statement)

Updated for more dynamic generation

In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from faker import Faker

# Initialize Faker
fake = Faker()

# Sample data with placeholders
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 1, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2, 'order_date': '2022-01-15'}
    ]
}

# Create DataFrames for the sample data
customers_df = pd.DataFrame(sample_data['customers'])
orders_df = pd.DataFrame(sample_data['orders'])

# Assign a 'name' attribute to DataFrames to represent table names (Updated)
customers_df.name = 'customers'
orders_df.name = 'orders'

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token for GPT-2

# Updated SQLDataset class to make SQL generation dynamic
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        table_name = self.df.name  # Dynamic table name (Updated)
        columns = row.index.tolist()  # Get column names from the DataFrame (Updated)
        values = []

        # Generate fake data for each column dynamically (Updated)
        for column in columns:
            if 'name' in column.lower():
                values.append(f"'{fake.name()}'")
            elif 'email' in column.lower():
                values.append(f"'{fake.email()}'")
            elif 'date' in column.lower():
                values.append(f"'{fake.date()}'")
            elif 'id' in column.lower():
                values.append(str(fake.random_int(min=1, max=1000)))
            else:
                values.append(f"'{fake.word()}'")  # Generic replacement for other columns

        # Create the SQL insert statement dynamically (Updated)
        input_text = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"

        # Tokenize the input text
        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
        return {'input_ids': inputs['input_ids'].squeeze(), 'attention_mask': inputs['attention_mask'].squeeze()}

# Combine datasets and create DataLoader with padding
combined_df = pd.concat([customers_df, orders_df], ignore_index=True)
dataset = SQLDataset(combined_df, tokenizer)

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_masks}

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 60
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.clone()  # GPT-2 generates the next token

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader)}')

model.eval()

# Generate SQL statements based on the prompt
def generate_sql_data(prompt, num_statements=20):
    generated_statements = []
    for _ in range(num_statements):
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], max_length=100, num_return_sequences=1)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Define patterns to identify column types dynamically (Updated)
        name_pattern = re.compile(r"\bname\b", re.IGNORECASE)
        email_pattern = re.compile(r"\bemail\b", re.IGNORECASE)
        date_pattern = re.compile(r"\bdate\b", re.IGNORECASE)
        id_pattern = re.compile(r"\bid\b", re.IGNORECASE)

        # Post-process to replace 'nan' or invalid data with realistic fake data dynamically (Updated)
        def replace_with_fake_data(match):
            column_name = match.group(0).lower()
            if name_pattern.search(column_name):
                return f"'{fake.name()}'"
            elif email_pattern.search(column_name):
                return f"'{fake.email()}'"
            elif date_pattern.search(column_name):
                return f"'{fake.date()}'"
            elif id_pattern.search(column_name):
                return str(fake.random_int(min=1, max=1000))
            else:
                return f"'{fake.word()}'"  # Generic replacement for other columns

        # Replace any 'nan' or invalid data by matching with column names (Updated)
        generated_sql = re.sub(r"'nan'", replace_with_fake_data, generated_sql)
        generated_sql = re.sub(r"'\d+'", replace_with_fake_data, generated_sql)

        generated_statements.append(generated_sql)

    return generated_statements

# Example usage:
prompt = "INSERT INTO customers (id, name, email) VALUES (1, 'nan', 'nan');"
sample_sql_statements = generate_sql_data(prompt, num_statements=20)

# Print the generated SQL statements
for statement in sample_sql_statements:
    print(statement)


Epoch 1/60, Loss: 3.3150335550308228
Epoch 2/60, Loss: 3.0263644456863403
Epoch 3/60, Loss: 2.67734158039093
Epoch 4/60, Loss: 2.312158703804016
Epoch 5/60, Loss: 2.1938542127609253
Epoch 6/60, Loss: 1.8724486827850342
Epoch 7/60, Loss: 1.7022778987884521
Epoch 8/60, Loss: 1.7141201496124268
Epoch 9/60, Loss: 1.5675982236862183
Epoch 10/60, Loss: 1.3163073658943176
Epoch 11/60, Loss: 1.144919216632843
Epoch 12/60, Loss: 1.138761281967163
Epoch 13/60, Loss: 0.99598827958107
Epoch 14/60, Loss: 0.9684582352638245
Epoch 15/60, Loss: 1.0192618072032928
Epoch 16/60, Loss: 0.9338520169258118
Epoch 17/60, Loss: 1.017622172832489
Epoch 18/60, Loss: 0.917255699634552
Epoch 19/60, Loss: 0.8865472078323364
Epoch 20/60, Loss: 0.9768170118331909
Epoch 21/60, Loss: 0.8881572782993317
Epoch 22/60, Loss: 0.8067397773265839
Epoch 23/60, Loss: 0.8851421773433685
Epoch 24/60, Loss: 0.7968523800373077
Epoch 25/60, Loss: 0.8314014375209808
Epoch 26/60, Loss: 0.7426825761795044
Epoch 27/60, Loss: 0.733972787

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Epoch 60/60, Loss: 0.7245734333992004


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

INSERT INTO customers (id, name, email) VALUES (1, 'staff', 'direction');

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
INSERT INTO customers (id, name, email) VALUES (1, 'agreement', 'field');

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
INSERT INTO customers (id, name, email) VALUES (1, 'ask', 'before');

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76
INSERT INTO customers (id, name, email) VALUES (1, 'safe', 'work');

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 

# Above is failed implementation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install faker

In [ ]:
import os
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset, DataLoader
from faker import Faker
import re  # Import the `re` module

# Initialize Faker
fake = Faker()
Faker.seed(4321)

# Sample data with placeholders
sample_data = {
    'customers': [
        {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'},
        {'id': 2, 'name': 'Jane Doe', 'email': 'jane.doe@example.com'}
    ],
    'orders': [
        {'id': 1, 'customer_id': 10001, 'order_date': '2022-01-01'},
        {'id': 2, 'customer_id': 2001, 'order_date': '2023-01-15'},
        {'id': 34, 'customer_id': 101, 'order_date': '2013-01-11'}
    ],
    'product': [
        {'skuid': '1231', 'desc': "electronics", 'inventory_count': 100, 'refill_date':'2024-01-01'},
        {'skuid': '2a23', 'desc': "furniture", 'inventory_count': 568 , 'refill_date':'2024-01-01'},
        {'skuid': '3awsd4', 'desc': "miscellaneous", 'inventory_count': 34562,'refill_date':'2024-01-01'}
    ]
}

# Create DataFrames for the sample data
customers_df = pd.DataFrame(sample_data['customers'])
orders_df = pd.DataFrame(sample_data['orders'])
product_df = pd.DataFrame(sample_data['product'])

# Assign a 'name' attribute to DataFrames to represent table names
customers_df.name = 'customers'
orders_df.name = 'orders'
product_df.name = 'product'

# Load the pre-trained GPT-2 model and tokenizer
model_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token  # Set pad_token to eos_token for GPT-2
model.config.pad_token_id = model.config.eos_token_id

# SQLDataset class for dynamic SQL generation
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        table_name = self.df.name  # Dynamic table name
        columns = row.index.tolist()  # Get column names from the DataFrame
        values = []

        # Generate fake data for each column dynamically
        for column in columns:
            if 'name' in column.lower():
                values.append(f"'{fake.name()}'")
            elif 'skuid' in column.lower():
                values.append(f"'{fake.bothify(letters='aedhfcwsqikdr')}'")
            elif 'email' in column.lower():
                values.append(f"'{fake.email()}'")
            elif 'date' in column.lower():
                values.append(f"'{fake.date()}'")
            elif 'desc' in column.lower():
                values.append(f"'{fake.paragraph(nb_sentences=1)}'")
            elif 'id' in column.lower():
                values.append(str(fake.random_int(min=1, max=1000)))
            elif 'inventory_count' in column.lower():
                values.append(str(fake.random_int(min=1, max=1000)))
            else:
                values.append(f"'{fake.word()}'")  # Generic replacement for other columns

        # Create the SQL insert statement dynamically
        input_text = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"

        # Tokenize the input text
        inputs = self.tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)
        return {'input_ids': inputs['input_ids'].squeeze(), 'attention_mask': inputs['attention_mask'].squeeze()}

# Combine datasets and create DataLoader with padding
combined_df = pd.concat([customers_df, orders_df, product_df], ignore_index=True)
dataset = SQLDataset(combined_df, tokenizer)

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)

    return {'input_ids': input_ids, 'attention_mask': attention_masks}

data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Fine tuning - Training the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.clone()  # GPT-2 generates the next token

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(data_loader)}')



# Save the model's state dictionary and tokenizer
model_save_path = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/SqlGenerator/gpt2_sql_generator.pth'
tokenizer_save_path = '/content/drive/MyDrive/Colab Notebooks/NLP/SavedModels/SqlGenerator/gpt2_tokenizer'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

torch.save(model.state_dict(), model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")
print(f"Tokenizer saved to {tokenizer_save_path}")

model.eval()

# Generate SQL statements based on the prompt
def generate_sql_data(prompt, num_statements=20):
    generated_statements = []
    for _ in range(num_statements):
        # Tokenize the input prompt
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(device)

        # Create the attention mask
        inputs['attention_mask'] = (inputs['input_ids'] != tokenizer.pad_token_id).long()

        # ignore attention for padding characters

        # Generate the output
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # Pass attention_mask here
            max_length=1000,
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id  # Ensure the model knows about pad_token_id
        )

        # outputs = model.generate(inputs['input_ids'], max_length=100, num_return_sequences=1)
        generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Decode and print the output for debugging
        print("Raw Generated Output:", generated_sql)  # Debugging line

        # Define patterns to identify column types dynamically
        name_pattern = re.compile(r"\bname\b", re.IGNORECASE)
        email_pattern = re.compile(r"\bemail\b", re.IGNORECASE)
        date_pattern = re.compile(r"\bdate\b", re.IGNORECASE)
        id_pattern = re.compile(r"\bid\b", re.IGNORECASE)
        desc_pattern = re.compile(r"\bdesc\b", re.IGNORECASE)
        sku_pattern = re.compile(r"\bsku\b", re.IGNORECASE)
        ssn_pattern = re.compile(r"\bssn\b", re.IGNORECASE)

        # Post-process to replace 'nan' or invalid data with realistic fake data
        def replace_with_fake_data(match):
            column_name = match.group(0).lower()
            if name_pattern.search(column_name):
                return f"'{fake.name()}'"
            elif email_pattern.search(column_name):
                return f"'{fake.email()}'"
            elif date_pattern.search(column_name):
                return f"'{fake.date()}'"
            elif id_pattern.search(column_name):
                return str(fake.random_int(min=1, max=1000))
            elif desc_pattern.search(column_name):
                return f"'{fake.paragraph(nb_sentences=1)}'"
            elif sku_pattern.search(column_name):
                return str(fake.bothify(letters='aedhfcwsqikdr'))
            elif ssn_pattern.search(column_name):
                return str(fake.ssn())
            else:
                return f"'{fake.word()}'"  # Generic replacement for other columns

        # Replace any 'nan' or invalid data by matching with column names
        generated_sql = re.sub(r"'nan'", replace_with_fake_data, generated_sql)

        generated_statements.append(generated_sql)

    return generated_statements


Epoch 1/100, Loss: 2.7288522720336914
Epoch 2/100, Loss: 2.306886315345764
Epoch 3/100, Loss: 1.9932240545749664
Epoch 4/100, Loss: 1.804379642009735
Epoch 5/100, Loss: 1.6513957977294922
Epoch 6/100, Loss: 1.375845581293106
Epoch 7/100, Loss: 1.3221988081932068
Epoch 8/100, Loss: 1.2304116785526276
Epoch 9/100, Loss: 1.1590915024280548
Epoch 10/100, Loss: 1.1357654333114624
Epoch 11/100, Loss: 1.0799949318170547
Epoch 12/100, Loss: 1.03450246155262
Epoch 13/100, Loss: 1.0222009420394897
Epoch 14/100, Loss: 0.9552304893732071
Epoch 15/100, Loss: 0.955097883939743
Epoch 16/100, Loss: 0.9440721273422241
Epoch 17/100, Loss: 0.9252956509590149
Epoch 18/100, Loss: 0.8740400671958923
Epoch 19/100, Loss: 0.9478059411048889
Epoch 20/100, Loss: 0.9007769972085953
Epoch 21/100, Loss: 0.853043720126152
Epoch 22/100, Loss: 0.9251676052808762
Epoch 23/100, Loss: 0.8781739920377731
Epoch 24/100, Loss: 0.8630498498678207
Epoch 25/100, Loss: 0.8842781484127045
Epoch 26/100, Loss: 0.8767949491739273
Ep

In [ ]:
# Example usage:
prompt = "INSERT INTO person (id, name, email, description)"
sample_sql_statements = generate_sql_data(prompt, num_statements=20)

# Print the generated SQL statements
for statement in sample_sql_statements:
    print(statement)

Raw Generated Output: INSERT INTO person (id, name, email, description) VALUES (9, 'Michael Brown','michael@example.org', 5, '2008-04-24', '20 hd', 'Military officer.', 5, '1970-08-24');
Raw Generated Output: INSERT INTO person (id, name, email, description) VALUES (9, 'Michael Brown','michael@example.org', 5, '2008-04-24', '20 hd', 'Military officer.', 5, '1970-08-24');
Raw Generated Output: INSERT INTO person (id, name, email, description) VALUES (9, 'Michael Brown','michael@example.org', 5, '2008-04-24', '20 hd', 'Military officer.', 5, '1970-08-24');
Raw Generated Output: INSERT INTO person (id, name, email, description) VALUES (9, 'Michael Brown','michael@example.org', 5, '2008-04-24', '20 hd', 'Military officer.', 5, '1970-08-24');
Raw Generated Output: INSERT INTO person (id, name, email, description) VALUES (9, 'Michael Brown','michael@example.org', 5, '2008-04-24', '20 hd', 'Military officer.', 5, '1970-08-24');
Raw Generated Output: INSERT INTO person (id, name, email, descri

In [ ]:
prompt = "create table Employee "
sample_sql_statements = generate_sql_data(prompt, num_statements=1)

# Print the generated SQL statements
for statement in sample_sql_statements:
    print(statement)

Raw Generated Output: create table Employee ( name, email, customer_id, order_date, skuid, desc, inventory_count, refill_date) VALUES (821, 'Michael Brown','michael@example.org', 597, '1986-03-24', '20 hd', 'Carry out a person.', 597, '2023-06-24');
create table Employee ( name, email, customer_id, order_date, skuid, desc, inventory_count, refill_date) VALUES (821, 'Michael Brown','michael@example.org', 597, '1986-03-24', '20 hd', 'Carry out a person.', 597, '2023-06-24');
